## Create a sample database

The schema was borrowed from this medium article https://medium.com/snowflake/a-sales-assistant-powered-by-a-react-agent-and-llama-2-on-snowflake-95f6331d43db.

In [1]:
database_schema = """

DROP TABLE IF EXISTS PRODUCTS;
CREATE TABLE PRODUCTS (
    SKU INT,
    DESCRIPTION STRING
);
DROP TABLE IF EXISTS INVENTORY;
CREATE TABLE INVENTORY (
    SKU INT,
    QUANTITY INT,
    WAREHOUSE STRING
);

INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (100,'short sleeve cotton shirt blue');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (200,'long sleeve flanelle shirt red');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (300, 'Black Running Shoes size 11');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (400, 'Sweat Pants Grey Men');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (500, 'Crop top green Women');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (600, 'Long Skirt Blue');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (700, 'Summer hat kids Orange');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (800, 'Sunglasses Unisex black');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (900, 'Jeans Black');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (1000, 'Kaki cotton pants summer women');

INSERT INTO INVENTORY VALUES (100, 154,'New York');
INSERT INTO INVENTORY VALUES (200, 245, 'Seattle');
INSERT INTO INVENTORY VALUES (300, 364, 'San Francisco');
INSERT INTO INVENTORY VALUES (400, 421, 'New York');
INSERT INTO INVENTORY VALUES (500, 532, 'Seattle');
INSERT INTO INVENTORY VALUES (600, 678, 'New York');
INSERT INTO INVENTORY VALUES (700, 741, 'San Francisco');
INSERT INTO INVENTORY VALUES (800, 845, 'New York');
INSERT INTO INVENTORY VALUES (900, 987, 'Seattle');
INSERT INTO INVENTORY VALUES (1000, 149, 'San Francisco');
"""

In [2]:
import sqlite3

connection_obj = sqlite3.connect('inventory.db')
# cursor object
cursor_obj = connection_obj.cursor()
cursor_obj.executescript(database_schema)
# Test inserted records
cursor_obj.execute("SELECT * from INVENTORY") 
# fetch all the data 
print(cursor_obj.fetchall())
# Another query
cursor_obj.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor_obj.fetchall())
connection_obj.close()

[(100, 154, 'New York'), (200, 245, 'Seattle'), (300, 364, 'San Francisco'), (400, 421, 'New York'), (500, 532, 'Seattle'), (600, 678, 'New York'), (700, 741, 'San Francisco'), (800, 845, 'New York'), (900, 987, 'Seattle'), (1000, 149, 'San Francisco')]
[('PRODUCTS',), ('INVENTORY',)]


In [3]:
import os

class DBTool():
    
    def __init__(self, database):
        
        if not os.path.isfile(database):
            print("Database does not exist")
            raise ValueError("Database does not exist")
        try:
            self.connection = sqlite3.connect(database)
            self.cursor = self.connection.cursor()
        except:
           print(f"Unable to open{database}") 
           raise ValueError(f"unable to open {database}")

        
        self.table_cols   = {}
        self.__get_tbls()
        self.__get_cols()
        self.connection.close()
    
    def get_string_signature(self):
        return_string = """
        Table names and their columns are listed below \n
        """
        
        table_count = 1
        for table, columns in self.table_cols.items():
            return_string+= str(table_count) + "\n"
            return_string+= f"Table name:{table} \n"
            for idx, col_name in enumerate(columns):
                return_string+="\t" + str(idx+1) + f" {col_name} \n"
            table_count+=1
        
        return return_string
            
    
    def __get_tbls(self):
        self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = self.cursor.fetchall()
        for (table,) in tables:
            self.table_cols[table] = []
    
    def __get_cols(self):
        for table in self.table_cols.keys():
            columns = self.cursor.execute(f"PRAGMA table_info({table})")
            for column in columns:
                self.table_cols[table].append(column[1])


db_tool = DBTool("inventory.db")
table_schema = db_tool.get_string_signature()
print(table_schema)


        Table names and their columns are listed below 

        1
Table name:PRODUCTS 
	1 SKU 
	2 DESCRIPTION 
2
Table name:INVENTORY 
	1 SKU 
	2 QUANTITY 
	3 WAREHOUSE 



In [6]:
from adalflow.core.generator import Generator
from adalflow.components.model_client.ollama_client import OllamaClient
from adalflow.core.default_prompt_template import SIMPLE_DEFAULT_LIGHTRAG_SYSTEM_PROMPT


db_template="""
<SYS>You are sqlite database expert. Given the list of tables and columns, you job is to translate
user queries into corresponding error free SQL statements. 
1. Make sure the column names in the query are from the table.
2. Think step by step and carefully check the user question, list of tables and column names before
giving the query back.
{% if table_schema_str %}
{{table_schema_str}}
{% endif %}
- Dont give any suggestion. Only generate the SQL.
</SYS>
Translate the input: {{input_str}}
You:
"""
host = "127.0.0.1:11434"

generator = Generator(
    model_client = OllamaClient(host=host)
   ,model_kwargs = {"model":"phi3:latest","options":{"temperature":0.7,"seed":77}}
   ,template = db_template
   ,prompt_kwargs ={"table_schema_str": table_schema}
   ,name ="SQL Generator" 
)

from adalflow.core.component import Component

class NLToSql(Component):
    
    def __init__(self, generator):
        super().__init__()
        self.generator = generator
        
    def call(self, input: str) -> str:
        input_dict = {"input_str":str(input)}
        result = self.generator.call(input_dict)
        return {"result": result.data, "query": str(input)}
        
nl_sql = NLToSql(generator)
query = "what is the inventory for product jeans black"
print(nl_sql(query)['result'])

SELECT i.SKU, p.DESCRIPTION, i.QUANTITY, i.WAREHOUSE 
FROM PRODUCTS AS p 
JOIN INVENTORY AS i ON p.SKU = i.SKU 
WHERE p.DESCRIPTION LIKE '%jeans black%';


In [17]:
from adalflow.core.component import fun_to_component
import json

@fun_to_component
def execute_sql(input_json):
    """
    Execute the given SQL command
    """
    import sqlite3
    import json
    
    connection = sqlite3.connect("inventory.db")
    cursor_obj = connection.cursor()
    
    sql = input_json["result"]
    user_query = input_json["query"]
    
    cursor_obj.execute(sql)
    
    rows = cursor_obj.fetchall()
    return_records = [ {cursor_obj.description[i][0]:x for i,x in enumerate(row)} for row in rows] 
    
    connection.close()
    
    return json.dumps({"user_input": user_query, "sql":sql,"results": return_records})


summarizer_template="""
<SYS>You can tell stories from number. Help summarize the numerical
answer provided in an easy to understand manner. The numerical answer
is the result from a database query for the user question.
- Give a clear explanation in a couple of lines.
</SYS>
Summarize the input: 
user question: {{input_str}}
numerical answer: {{numerical_answer}}
You:
"""
host = "127.0.0.1:11434"

summary_generator = Generator(
    model_client = OllamaClient(host=host)
   ,model_kwargs = {"model":"phi3:latest","options":{"temperature":0.7,"seed":77}}
   ,template = summarizer_template
   ,name ="SQL Result Summarizer" 
)

class SQLSummarizer(Component):
    
    def __init__(self, generator):
        super().__init__()
        self.generator = summary_generator
        
    def call(self, input: str) -> str:
        
        input_dict = json.loads(input)
        sql = input_dict["sql"]
        user_query = input_dict["user_input"]
        numerics   = input_dict["results"]
        input_dict = {"input_str": user_query, "numerical_answer": numerics}
        result = self.generator.call(input_dict)
        return json.dumps({"result": result.data, "query": str(user_query), "sql": sql})
    

In [18]:
from adalflow.core.container import Sequential

summarizer = SQLSummarizer(summary_generator)

seq = Sequential(nl_sql, execute_sql, summarizer)
query = "what is the inventory for product jeans black"
print(seq(query))

{"result": "The Seattle warehouse has 987 pairs of black jeans with SKU number 900 in stock.", "query": "what is the inventory for product jeans black", "sql": "SELECT i.SKU, p.DESCRIPTION, i.QUANTITY, i.WAREHOUSE \nFROM PRODUCTS AS p \nJOIN INVENTORY AS i ON p.SKU = i.SKU \nWHERE p.DESCRIPTION LIKE '%jeans black%';"}
